# Part 1: Decision trees
* *There are two main kinds of decision trees depending on the type of output (numeric vs. categorical). What are they?*
* *Explain in your own words: Why is entropy useful when deciding where to split the data?*
* *Why are trees prone to overfitting?*
* *Explain (in your own words) how random forests help prevent overfitting.*

***[ANSWERS TO QUESTIONS]***

Loading the dataset, before we begin:

In [47]:
import requests
from matplotlib import pyplot as plt
import numpy as np
import csv
import pandas as pd
from pandas import DataFrame
%matplotlib inline
import geoplotlib
from geoplotlib.utils import BoundingBox
from geoplotlib.colors import ColorMap
import sklearn
from sklearn.tree import DecisionTreeClassifier,export_graphviz
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random as rn

In [19]:
# Load it into a Dataframe using pandas
path = '..\data\sfpd_incidents.csv'
df = pd.read_csv(path)
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,150060275,NON-CRIMINAL,LOST PROPERTY,Monday,01/19/2015,14:00,MISSION,NONE,18TH ST / VALENCIA ST,-122.421582,37.761701,"(37.7617007179518, -122.42158168137)",15006027571000
1,150098210,ROBBERY,"ROBBERY, BODILY FORCE",Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821003074
2,150098210,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821004014
3,150098210,SECONDARY CODES,DOMESTIC VIOLENCE,Sunday,02/01/2015,15:45,TENDERLOIN,NONE,300 Block of LEAVENWORTH ST,-122.414406,37.784191,"(37.7841907151119, -122.414406029855)",15009821015200
4,150098226,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Tuesday,01/27/2015,19:00,NORTHERN,NONE,LOMBARD ST / LAGUNA ST,-122.431119,37.800469,"(37.8004687042875, -122.431118543788)",15009822628160


*The chief wants you to start from real data and build a system that replicates the functionality in the Minority Report system. Imagine, we find out that certain type of crime is going to take place - as well as the exact time of the crime - but that we don't know where, then Suneman wants an algorithm that will predict which district the crime is most likely to take place in. Specifically, let's build an algorithm that predicts the location of a crime based on its type and time.*

* *Use the category of the crimes to build a decision tree that predicts the corresponding district. You can implement the ID3 tree in the DSFS book, or use the [DecisionTreeClassifier](http://scikit-learn.org/stable/modules/tree.html) class in scikit-learn. For training, you can use 90% of the data and test the tree prediction on the remaining 10%.*

In order to fit the classifier, we must transform the data and convert all the string attributes to categorical attributes, since they're not supported by the DecisionTreeClassifier. Therefore, we use **one-out-of-K** coding in order to associate to each category its numerical representation. `scikit-learn` already provides us with a class made for this purpose, the `LabelEncoder`.

With `train_test_split` we're automatically splitting the whole dataset in training/test sets with the ratio suggested by the instructions. We hereby define the functions that will be useful later on:

In [39]:
# Split training and testing data
def split_dataset(X,Y):
    return train_test_split(X, Y, test_size=0.1, random_state=0)

#Train classifier and predict labels
def train_predict_tree_clf(X_train,Y_train,Y_test):
    #Fit the classifier
    clf = DecisionTreeClassifier()
    clf.fit(X_train[:,None],Y_train[:,None])
    return clf.predict(Y_test[:,None])

In [40]:
#Encoding categorical attributes
label_encoder = LabelEncoder()
X = label_encoder.fit_transform(df['Category'])
Y = label_encoder.fit_transform(df['PdDistrict'])
 
X_train, X_test, Y_train, Y_test = split_dataset(X,Y)
pred = train_predict_tree_clf(X_train,Y_train,Y_test)

We re-convert the numerical values of `Y_test` and `pred` to categorical, since we'll need them for the next step.

In [41]:
pred = label_encoder.inverse_transform(pred)
Y_test = label_encoder.inverse_transform(Y_test)

* *What is the fraction of correct predictions?*

Let's create a function to compare the actual values with the predicted values and calculate the fraction of correct predictions as the ratio between the number of correct predictions and the length of the test set:

In [42]:
#Given the Y_test vector, the pred vector and the name of the predicted label, calculate the correctness ratio
def calculate_ratio(Y_test,pred,label,district_name=None):
    #Support dataframe with columns to compare
    df_predictions = DataFrame(np.column_stack([Y_test,pred]),columns=[label+'_Real',label+'_Pred'])
    total = len(df_predictions.index)
    
    #Calculate ratio
    num_correct_predictions = len(df_predictions[df_predictions[label+'_Real']==df_predictions[label+'_Pred']].index)
    ratio = num_correct_predictions*100.0/total
    district = '' if district_name==None else 'for district '+district_name
    print 'Fraction of correct predictions %s: %.2f%%' %(district,ratio)

And calculate the correctness ratio of our first prediction:

In [43]:
calculate_ratio(Y_test,pred,'PdDistrict')

Fraction of correct predictions : 18.03%


According to the output above, we guess the classifier didn't perform so well if we keep in consideration all the districts.

* *What are the correct predictions if you restrict the training/prediction to single districts (for example, predicting Mission vs. all other districts, etc)?*

We're supposed to perform a **Binary Classification**, by picking one district (e.g. **MISSION**) and labeling all other districts as **NOT MISSION**. Therefore, we'll have only *two values* for the labels (ideally 0 and 1, that's why *binary*) We'll manually convert to the string notation and let the Label Encoder do the rest.

In [45]:
#Change this in order to change the selected district
#Just some example districts
districts = ['SOUTHERN','MISSION','TENDERLOIN','NORTHERN']

#Function to get the binary representation of a specific value
def get_binary_repr(value,district):
    if value==district:
        return district
    return 'NOT'+district

for district in districts:
    df_binary_districts = df
    df_binary_districts['PdDistrict_Bin'] = df_binary_districts['PdDistrict'].apply(lambda x: get_binary_repr(x,district))

    # Split training and testing data (N.B. X is the same as before)
    Y = df_binary_districts['PdDistrict_Bin']
    Y = label_encoder.fit_transform(Y)
    X_train, X_test, Y_train, Y_test = split_dataset(X,Y)
    
    #Train classifier and predict
    pred = train_predict_tree_clf(X_train,Y_train,Y_test)

    #Again re-converting to categorical values
    pred = label_encoder.inverse_transform(pred)
    Y_test = label_encoder.inverse_transform(Y_test)

    #Calculate fraction of correct predictions
    calculate_ratio(Y_test,pred,'PdDistrict',district)

Fraction of correct predictions for district SOUTHERN: 81.97%
Fraction of correct predictions for district MISSION: 86.62%
Fraction of correct predictions for district TENDERLOIN: 91.14%
Fraction of correct predictions for district NORTHERN: 87.74%


In light of the above results, we could definitely say that reducing the problem to a binary classification problem makes more sense in terms of correct predictions.

* *Compare it to the random guess, what would you get if you'd guess a district randomly?*

***NOT CORRECT***

In [55]:
all_districts = list(set(df['PdDistrict']))

def random_guess():
    return rn.choice(all_districts)

df_random_guess = df
df_random_guess['PdDistrict_Rn'] = pd.Series(np.array([random_guess() for _ in range(len(df_random_guess.index))]))

# Split training and testing data (N.B. X is the same as before)
Y = df_random_guess['PdDistrict_Rn']
Y = label_encoder.fit_transform(Y)
X_train, X_test, Y_train, Y_test = split_dataset(X,Y)

#Train classifier and predict
pred = train_predict_tree_clf(X_train,Y_train,Y_test)

#Again re-converting to categorical values
pred = label_encoder.inverse_transform(pred)
Y_test = label_encoder.inverse_transform(Y_test)

#Calculate fraction of correct predictions
print '--- Random guess ---'
calculate_ratio(Y_test,pred,'PdDistrict')

--- Random guess ---
Fraction of correct predictions : 18.16%


* *And if you'd guess always one of the districts (for example the district with the most crimes)?*